In [ ]:
import pandas as pd, numpy as np, lightgbm as lgb, shap
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load data
df = pd.read_parquet("influencer_modelling_ready.parquet")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

In [ ]:
# Create ROI calculation with available columns
Q, E = df["Quality_Audience"], df["ER"]
M = df["Turkey"]/100
P = df["Est_Post_Price"]

# Use Comment_Rate as CM proxy (influence multiplier)
CM = df["Comment_Rate"]
IM = 1.0  # Placeholder for influence multiplier

# Calculate ROI_star
df["ROI_star"] = (Q * E * M * IM * (1 + 0.2*CM)) / P

# Standardize for modeling
y = (df["ROI_star"] - df["ROI_star"].mean()) / df["ROI_star"].std()

print(f"ROI_star stats: mean={df['ROI_star'].mean():.2f}, std={df['ROI_star'].std():.2f}")
print(f"Target y stats: mean={y.mean():.2f}, std={y.std():.2f}")

In [ ]:
# ---------- TRAIN / VALIDATION SPLIT ----------
# Remove ROI_star from features (don't want to predict using target)
feature_cols = [col for col in df.columns if col != "ROI_star"]
X = df[feature_cols]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ---------- LIGHTGBM REGRESSOR ----------
reg = lgb.LGBMRegressor(
    objective="regression", 
    metric="rmse",
    learning_rate=0.03, 
    n_estimators=800,
    num_leaves=128, 
    min_data_in_leaf=10,
    subsample=0.8, 
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1  # Suppress training output
)

# Train model
reg.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.log_evaluation(period=0)]  # Suppress eval logs
)

print(f"Training score: {reg.score(X_train, y_train):.4f}")
print(f"Validation score: {reg.score(X_val, y_val):.4f}")

# ---------- SHAP ANALYSIS ----------
try:
    explainer = shap.TreeExplainer(reg)
    shap_values = explainer.shap_values(X_val.iloc[:100])  # Use subset for speed
    
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_val.iloc[:100], max_display=25, show=False)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"SHAP analysis failed: {e}")
    print("Showing feature importance instead:")
    
    # Fallback to feature importance
    importance = reg.feature_importances_
    features = X.columns
    
    # Sort and display top features
    feature_imp = pd.DataFrame({
        'feature': features,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 Feature Importances:")
    print(feature_imp.head(15))

In [ ]:
# ---------- STATISTICAL SIGNIFICANCE TESTS ----------
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

print("=" * 60)
print("STATISTICAL SIGNIFICANCE ANALYSIS")
print("=" * 60)

# 1. Model Performance Significance Tests
y_pred = reg.predict(X_val)
residuals = y_val - y_pred

# Test if residuals are normally distributed (assumption for many tests)
shapiro_stat, shapiro_p = stats.shapiro(residuals[:5000] if len(residuals) > 5000 else residuals)
print(f"\n1. RESIDUAL NORMALITY TEST (Shapiro-Wilk):")
print(f"   Statistic: {shapiro_stat:.4f}, p-value: {shapiro_p:.4e}")
print(f"   Residuals {'ARE' if shapiro_p > 0.05 else 'ARE NOT'} normally distributed (α=0.05)")

# Test if model predictions are significantly different from random
random_pred = np.random.normal(y_val.mean(), y_val.std(), len(y_val))
t_stat, t_p = stats.ttest_ind(y_pred, random_pred)
print(f"\n2. MODEL vs RANDOM PREDICTIONS (T-test):")
print(f"   T-statistic: {t_stat:.4f}, p-value: {t_p:.4e}")
print(f"   Model predictions {'ARE' if t_p < 0.05 else 'ARE NOT'} significantly different from random (α=0.05)")

# 2. Feature-Target Correlation Significance Tests
print(f"\n3. FEATURE-TARGET CORRELATION TESTS:")
print("   Top 10 features with significant correlations:")

correlations = []
for col in X.columns:
    if X[col].dtype in ['int64', 'float64']:
        corr_coef, corr_p = stats.pearsonr(X[col], y)
        correlations.append({
            'feature': col,
            'correlation': corr_coef,
            'p_value': corr_p,
            'significant': corr_p < 0.05
        })

corr_df = pd.DataFrame(correlations).sort_values('correlation', key=abs, ascending=False)
print(corr_df.head(10).to_string(index=False))

# 3. Feature Importance Permutation Test
print(f"\n4. PERMUTATION IMPORTANCE TEST:")
print("   Testing if feature importance is statistically significant...")

perm_importance = permutation_importance(
    reg, X_val, y_val, n_repeats=10, random_state=42, scoring='r2'
)

# Create dataframe with permutation results
perm_df = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
})

# Calculate z-scores for importance (assuming normal distribution)
perm_df['z_score'] = perm_df['importance_mean'] / (perm_df['importance_std'] + 1e-10)  # Add small epsilon to avoid division by zero
perm_df['p_value'] = 2 * (1 - stats.norm.cdf(abs(perm_df['z_score'])))
perm_df['significant'] = perm_df['p_value'] < 0.05
perm_df = perm_df.sort_values('importance_mean', ascending=False)

print("   Top 10 features with permutation importance:")
print(perm_df.head(10)[['feature', 'importance_mean', 'p_value', 'significant']].to_string(index=False))

# 4. ROI Component Significance Tests
print(f"\n5. ROI COMPONENTS SIGNIFICANCE TESTS:")
roi_components = {
    'Quality_Audience': df['Quality_Audience'],
    'ER': df['ER'],
    'Turkey': df['Turkey'],
    'Comment_Rate': df['Comment_Rate'],
    'Est_Post_Price': df['Est_Post_Price']
}

print("   Individual component correlations with ROI_star:")
for name, component in roi_components.items():
    corr_coef, corr_p = stats.pearsonr(component, df['ROI_star'])
    print(f"   {name:15}: r={corr_coef:7.4f}, p={corr_p:.4e} {'***' if corr_p < 0.001 else '**' if corr_p < 0.01 else '*' if corr_p < 0.05 else ''}")

# 5. Model Validation Tests
print(f"\n6. MODEL VALIDATION TESTS:")

# Cross-validation significance test
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='r2')
cv_mean, cv_std = cv_scores.mean(), cv_scores.std()

# Test if CV scores are significantly > 0
t_stat_cv, p_val_cv = stats.ttest_1samp(cv_scores, 0)
print(f"   Cross-validation R² scores: {cv_mean:.4f} ± {cv_std:.4f}")
print(f"   T-test vs 0: t={t_stat_cv:.4f}, p={p_val_cv:.4e}")
print(f"   Model performance {'IS' if p_val_cv < 0.05 else 'IS NOT'} significantly better than baseline (α=0.05)")

# 6. Outlier Detection and Significance
print(f"\n7. OUTLIER ANALYSIS:")
z_scores = np.abs(stats.zscore(y))
outliers = np.where(z_scores > 3)[0]
print(f"   Number of outliers (|z-score| > 3): {len(outliers)} ({len(outliers)/len(y)*100:.1f}%)")

if len(outliers) > 0:
    # Test if removing outliers significantly improves model
    X_no_outliers = X.drop(outliers)
    y_no_outliers = y.drop(outliers)
    
    reg_no_outliers = lgb.LGBMRegressor(
        objective="regression", metric="rmse", learning_rate=0.03, 
        n_estimators=400, random_state=42, verbose=-1
    )
    
    X_train_clean, X_val_clean, y_train_clean, y_val_clean = train_test_split(
        X_no_outliers, y_no_outliers, test_size=0.2, random_state=42
    )
    
    reg_no_outliers.fit(X_train_clean, y_train_clean)
    r2_clean = reg_no_outliers.score(X_val_clean, y_val_clean)
    r2_original = reg.score(X_val, y_val)
    
    print(f"   R² with outliers: {r2_original:.4f}")
    print(f"   R² without outliers: {r2_clean:.4f}")
    print(f"   Improvement: {r2_clean - r2_original:.4f}")

print(f"\n" + "=" * 60)
print("SUMMARY OF STATISTICAL SIGNIFICANCE")
print("=" * 60)
print(f"• Model significantly outperforms random: {'✓' if t_p < 0.05 else '✗'}")
print(f"• Cross-validation significantly > 0: {'✓' if p_val_cv < 0.05 else '✗'}")
print(f"• Residuals are normally distributed: {'✓' if shapiro_p > 0.05 else '✗'}")
print(f"• Significant features found: {sum(perm_df['significant'])}/{len(perm_df)}")
print(f"• Significant ROI correlations: {sum(corr_df['significant'])}/{len(corr_df)}")